# Mesure de l'incertitude d'un modèle de régression avec MAPIE
---

Dans ce TP, nous allons estimer des intervalles de prédiction avec MAPIE.

Nous allons pouvoir déterminer la validité de nos intervalles de prédiction via deux métriques :

- la couverture "effective", qui est le pourcentage de données de test incluses dans les intervalles de prédiction. Par exemple, pour un niveau de confiance cible de 90%, il faut que 90% des données de test soit comprises dans les intervalles produits.
- la largeur moyenne des intervalles de prédiction qui doit être la plus proche possible de la largeur "théorique" ayant servi à générer le bruit de données.

# Import

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, QuantileRegressor
from mapie.metrics import regression_coverage_score, regression_mean_width_score
from mapie_v1.regression import CrossConformalRegressor, ConformalizedQuantileRegressor

from utils.dataset import (
    x_sinx,
    get_1d_data_with_constant_noise,
    get_1d_data_with_heteroscedastic_noise,
    get_1d_data_with_normal_distribution,
)
from utils.viz import (
    plot_regression,
    plot_uncertainties,
    plot_prediction_interval_width,
)

# L'incertitude en régression

## Bruit homoscédastique

Commençons ici par construire un jeu de données artificiel. Nous utiliserons la fonction $f(x) = x\sin(x)$ à laquelle nous ajoutons un bruit gaussien constant.

In [ ]:
X, y, X_test, y_test, y_mesh = get_1d_data_with_constant_noise(
    funct=x_sinx,
    min_x=-5,
    max_x=5,
    n_samples=600,
    noise=0.5
)

Visualisons le jeu de données et leur fonction génératrice.

In [ ]:
plot_regression(
    X_test,
    y_test,
    y_mesh,
    name_mesh="Génératrice",
    title="Problème homoscédastique",
)

Nous allons apprend un modèle polynomial pour ajuster les données.

In [ ]:
polyn_model = Pipeline(
    [
        ("poly", PolynomialFeatures(degree=10)),
        ("linear", LinearRegression())
    ]
)

**Exercice 1** : On souhaite à présent entraîner ce modèle via MAPIE et obtenir des intervalles de confiance à 95%.
- Instancier un `CrossConformalRegressor` enrobant notre modèle polynomial avec la méthode CV+ avec 5 plis de validation croisée.
- Entraîner et conformaliser le `CrossConformalRegressor` sur le jeu de données
- Prédire sur le jeu de test avec un niveau de risque de 5%

Visualisons les intervalles de prédictions obtenus sur le jeu de test.

In [ ]:
plot_uncertainties(
    X_test,
    y_test,
    y_preds,
    y_pred_intervals,
    title="Intervalles de prédiction avec niveau de confiance de 95%"
)

Visualisons la largeur de l'intervalle de prédictions en fonction de $x$.

In [ ]:
plot_prediction_interval_width(
    X_test,
    y_pred_intervals,
    title="Largeur des intervalles de prédictions",
    yaxis_title="Largeur"
)

Ici on voit que l'intervalle de confiance est à peu près constant, et c'est normal au vu de l'homoscédasticité du problème !

**Exercice 2** : calculer les métriques d'incertitudes :
- le taux de couverture (`regression_coverage_score`)
- la taille moyenne des intervalles de prédiction (`regression_mean_width_score`)
- a-t-on bien atteint le taux de couverture cible de 95% ?
- la taille théorique des intervalles est `1.96`. La taille moyenne des intervalles prédit par MAPIE est-elle supérieure ? Inférieure ?

## Bruit hétéroscédastique

Commençons ici par construire un jeu de données artificiel. Nous utiliserons la fonction $f(x) = x\sin(x)$ à laquelle nous ajoutons un bruit gaussien proportionnel à $x$.

In [ ]:
X, y, X_test, y_test, y_mesh = get_1d_data_with_heteroscedastic_noise(
    funct=x_sinx,
    min_x=0,
    max_x=5,
    n_samples=600,
    noise=0.5
)

Visualisons le jeu de données et leur fonction génératrice.

In [ ]:
plot_regression(
    X_test,
    y_test,
    y_mesh,
    name_mesh="Génératrice",
    title="Problème hétéroscédastique",
)

**Exercice 3** : On souhaite à présent entraîner ce modèle via MAPIE et obtenir des intervalles de confiance à 95%.
- Instancier un `CrossConformalRegressor` enrobant notre modèle polynomial avec la méthode CV+ avec 5 plis de validation croisée.
- Entraîner le `CrossConformalRegressor` sur le jeu de données
- Prédire sur le jeu de test avec un niveau de risque de 5%

Visualisons les intervalles de prédictions obtenus sur le jeu de test.

In [ ]:
plot_uncertainties(
    X_test,
    y_test,
    y_preds,
    y_pred_intervals,
    title="Intervalles de prédiction avec niveau de confiance de 95%"
)

Visualisons la largeur de l'intervalle de prédictions en fonction de $x$.

In [ ]:
plot_prediction_interval_width(
    X_test,
    y_pred_intervals,
    title="Largeur des intervalles de prédictions",
    yaxis_title="Largeur"
)

Ici on voit que l'intervalle de confiance est à peu près constant, et alors que le bruit dans les données ne l'est pas du tout !

**Exercice 4** : calculer les métriques d'incertitudes :
- le taux de couverture (`regression_coverage_score`)
- la taille moyenne des intervalles de prédiction (`regression_mean_width_score`)
- a-t-on bien atteint le taux de couverture cible de 95% ?

Ici, on voit que nos intervalle ne sont pas du tout adaptatifs ! Heureusement, il y a une solution : la régression quantile conforme. Instancions d'abord un modèle quantile.

In [ ]:
polyn_model_quant = Pipeline(
    [
        ("poly", PolynomialFeatures(degree=10)),
        ("linear", QuantileRegressor(solver="highs", alpha=0))
    ]
)

**Exercice 5** : On souhaite à présent entraîner ce modèle via MAPIE et obtenir des intervalles de confiance à 95%.
- Séparer les données d'entrée (`X` et `y`) en `X_train`, `X_conformalize`, `y_train`, `y_conformalize`
- Instancier un `ConformalizedQuantileRegressor` enrobant notre modèle polynomial
- Entraîner le `MapieQuantileRegressor` sur le jeu d'entraînement, et le conformaliser sur le jeu de conformalisation
- Prédire sur le jeu de test avec un niveau de risque de 5%

Visualisons les intervalles de prédictions obtenus sur le jeu de test.

In [ ]:
plot_uncertainties(
    X_test,
    y_test,
    y_preds,
    y_pred_intervals,
    title="Intervalles de prédiction avec niveau de confiance de 95%"
)

Visualisons la largeur de l'intervalle de prédictions en fonction de $x$.

In [ ]:
plot_prediction_interval_width(
    X_test,
    y_pred_intervals,
    title="Largeur des intervalles de prédictions",
    yaxis_title="Largeur"
)

Ah ça y est ! On a bien capturé l'hétéroscédacticité !

**Exercice 6** : calculer les métriques d'incertitudes :
- le taux de couverture (`regression_coverage_score`)
- la taille moyenne des intervalles de prédiction (`regression_mean_width_score`)
- a-t-on bien atteint le taux de couverture cible de 95% ?

Bingo ! Le taux de couverture est toujours bon, et en plus la taille moyenne de nos intervalles a largement diminué !

## Incertitude épistémique

Commençons ici par construire un jeu de données artificiel. Nous utiliserons la fonction $f(x) = x\sin(x)$ à laquelle nous ajoutons un bruit gaussien constant, mais dont les points de données sont répartis de manière inhomogène.

In [ ]:
X, y, X_test, y_test, y_mesh = get_1d_data_with_normal_distribution(
    funct=x_sinx,
    mu=0,
    sigma=2,
    n_samples=600,
    noise=0.5
)

Visualisons le jeu de données et leur fonction génératrice.

In [ ]:
plot_regression(
    X_test,
    y_test,
    y_mesh,
    name_mesh="Génératrice",
    title="Problème épistémique",
)

**Exercice 7** : On souhaite à présent entraîner ce modèle via MAPIE et obtenir des intervalles de confiance à 95%.
- Séparer les données d'entrée (`X` et `y`) en `X_train`, `X_conformalize`, `y_train`, `y_conformalize`
- Instancier un `ConformalizedQuantileRegressor` enrobant notre modèle polynomial
- Entraîner le `MapieQuantileRegressor` sur le jeu d'entraînement, et le conformaliser sur le jeu de conformalisation
- Prédire sur le jeu de test avec un niveau de risque de 5%

Visualisons les intervalles de prédictions obtenus sur le jeu de test.

In [ ]:
plot_uncertainties(
    X_test,
    y_test,
    y_preds,
    y_pred_intervals,
    title="Intervalles de prédiction avec niveau de confiance de 95%"
)

Visualisons la largeur de l'intervalle de prédictions en fonction de $x$.

In [ ]:
plot_prediction_interval_width(
    X_test,
    y_pred_intervals,
    title="Largeur des intervalles de prédictions",
    yaxis_title="Largeur"
)

On voit que les intervalles de confiance explosent quand la densité du jeu de données s'amoindrit, on a bien capté l'erreur épistémique !

**Exercice 8** : calculer les métriques d'incertitudes :
- le taux de couverture (`regression_coverage_score`)
- la taille moyenne des intervalles de prédiction (`regression_mean_width_score`)
- a-t-on bien atteint le taux de couverture cible de 95% ?

Bravo, vous maîtrisez les incertitudes en régression avec MAPIE !